<a href="https://colab.research.google.com/github/GMOD/jbrowse-jupyter/blob/main/skbr3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jbrowse-jupyter

In [2]:
from dash import html, Dash, dcc
from dash.dependencies import Input, Output
from jbrowse_jupyter import create, create_component

In [3]:
 app = Dash(__name__)
 server = app.server

In [4]:
# locations of interest and name
# sources: 
# https://jbrowse.org/demos/cancer-demo-2020/#embed
# https://genome.cshlp.org/content/early/2018/06/28/gr.231100.117.abstract
locations = [
    {"value": "NC_000014.8:50234871..50253103", "label": "KLHDC2"},
    {"value": "NC_000017.10:76670129..76778424", "label": "CYTH1"},
    {"value": "NC_000020.10:34213953..34252822", "label": "CPNE1"},
    {"value": "NC_000017.10:38060848..38074903", "label": "GSDMB"},
    {"value": "8:116962736..117337297", "label": "LINC00536"},
    {"value": "8:121547996..121824286", "label": "SNTB1"},
    {"value": "8:117654369..117768062", "label": "EIF3H"},
    {"value": "NC_000020.10:47240790..47444536", "label": "PREX1"},
    {"value": "8:125500735..125551329", "label": "TATDN1"},
    {"value": "8:128,806,779..129,113,499", "label": "PVT1"}
]

In [5]:
# track data, track name, track id
tracks = [
    ("https://s3.amazonaws.com/jbrowse.org/genomes/hg19/skbr3/reads_lr_skbr3.fa_ngmlr-0.2.3_mapped.splitters.bam", "SKBR3 pacbio (NGMLR) splitters only","ngmlr_splitters"),
    ("https://s3.amazonaws.com/jbrowse.org/genomes/hg19/skbr3/reads_lr_skbr3.fa_ngmlr-0.2.3_mapped.splitters.cram", "SKBR3 pacbio (NGMLR) splitters only (CRAM)","ngmlr_splitters_cram"),
    ("https://s3.amazonaws.com/jbrowse.org/genomes/hg19/skbr3/reads_lr_skbr3.fa_ngmlr-0.2.3_mapped.down.bam", "SKBR3 pacbio (NGMLR) 0.3 downsample","ngmlr_downsample"),
    ("https://s3.amazonaws.com/jbrowse.org/genomes/hg19/skbr3/reads_lr_skbr3.fa_ngmlr-0.2.3_mapped.down.cram", "SKBR3 pacbio (NGMLR) 0.3 downsample (CRAM)","reads_lr_skbr3.fa_ngmlr-0.2.3_mapped.down"),
    ("https://s3.amazonaws.com/jbrowse.org/genomes/hg19/reads_lr_skbr3.fa_ngmlr-0.2.3_mapped.bam.regions.bw", "SKBR3 pacbio coverage (NGMLR)","ngmlr_cov"),
    ("https://s3.amazonaws.com/jbrowse.org/genomes/hg19/reads_lr_skbr3.fa_ngmlr-0.2.3_mapped.bam.sniffles1kb_auto_l8_s5_noalt.vcf.gz", "SKBR3 pacbio (sniffles VCF)","breast_cancer_sniffles_hg19"),
    ("https://s3.amazonaws.com/jbrowse.org/genomes/hg19/skbr3/SKBR3_550bp_pcrFREE_S1_L001_AND_L002_R1_001.101bp.bwamem.ill.mapped.sort.bam", "SKBR3 illumina bwamem (BAM)","SKBR3 illumina bwamem (BAM)-id"),
    ("https://s3.amazonaws.com/jbrowse.org/genomes/hg19/skbr3/SKBR3_550bp_pcrFREE_S1_L001_AND_L002_R1_001.101bp.bwamem.ill.mapped.sort.cram", "SKBR3 illumina bwamem (CRAM)","SKBR3 illumina bwamem (CRAM)-id"),
    ("https://s3.amazonaws.com/jbrowse.org/genomes/hg19/reads_lr_skbr3.fa_ngmlr-0.2.3_mapped.bam", "SKBR3 pacbio (NGMLR)","ngmlr"),
    ("https://s3.amazonaws.com/jbrowse.org/genomes/hg19/GRCh37_latest_genomic.sort.gff.gz", "NCBI RefSeq (GFF3Tabix)","ncbi_gff_hg19")
]

In [6]:
conf_lgv = create("LGV", genome="hg19")
conf_cgv = create("CGV", genome="hg19")
for track in tracks:
    data = track[0]
    track_name = track[1]
    track_id = track[2]
    conf_lgv.add_track(data, name=track_name, track_id=track_id)
# adding a text search adapter for LGV 
ix = "https://jbrowse.org/genomes/hg19/trix/hg19.ix"
ixx = "https://jbrowse.org/genomes/hg19/trix/hg19.ixx"
meta = "https://jbrowse.org/genomes/hg19/trix/meta.json"
conf_lgv.add_text_search_adapter(ix, ixx, meta)
conf_lgv.set_default_session(['ncbi_gff_hg19'], False)
conf_cgv.add_track("https://s3.amazonaws.com/jbrowse.org/genomes/hg19/reads_lr_skbr3.fa_ngmlr-0.2.3_mapped.bam.sniffles1kb_auto_l8_s5_noalt.vcf.gz",name="SKBR3 pacbio (sniffles VCF)", track_id="breast_cancer_sniffles_hg19")
conf_cgv.set_default_session(["breast_cancer_sniffles_hg19"], False)
component_test = create_component(conf_cgv.get_config(), id="cgv-test", dash_comp="CGV")

In [7]:
# Return the component with a specific location.
@app.callback(
    Output("LGV-component", "children"),
    Input("location-to-view", "value")
)
def return_jbrowse(location):
    conf_lgv.set_location(location)
    return create_component(conf_lgv.get_config(), id="lgv-test", dash_comp="LGV")

In [8]:
app.layout = html.Div(
    [
        html.P(" SKBR3 Genes involved in fusions "),
        dcc.Dropdown(
            id="location-to-view", options=locations, value="1:229354402..229354403", placeholder="Select a gene to view."
        ),
        html.Hr(),
        dcc.Loading(id="LGV-component"),
        html.P("Circular Genome View with SKBR3 pacbio (sniffles VCF) track"),
        component_test
    ]
)

In [9]:
app.run(mode="inline", height=1000, use_reloader=False, debug=True, port=3031)
